In [ ]:
!ls dataset.zip || {wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V_a2wmNp3o02X2j0PONTtCCPCOn-kiJl" -O dataset.zip && rm -rf /tmp/cookies.txt}

In [ ]:
!ls dataset/ || unzip dataset.zip

In [ ]:
import pandas as pd
from preprocessing import *
from models import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [ ]:
NET_COLS = ["dport", "proto", "flags", "size", "modbus_fn", "n_pkt_src", "n_pkt_dst", "modbus_response"]

In [ ]:
net_attack = load_network_csv("dataset/Network datatset/csv/attack_1.csv")
phy_attack =` load_physical_csv("dataset/Physical dataset/phy_att_1.csv")

# Preprocessing:

## For the network dataset:
    - ignoring most of the cols except those above
    - Categorazing everything except size
## For the physical dataset:
    - Nothing special as everything is well defined

# Network Data

In [ ]:
# Getting a dataset with 20% being attacks:
pos = net_attack[(net_attack["label"] != 'anomaly') & (net_attack["label_n"] == 1)].iloc[:20000]
neg = net_attack[net_attack["label_n"] == 0].iloc[:80000]

In [ ]:
net_attack = pd.concat([pos, neg])

In [ ]:
contam = (net_attack["label_n"] == 1).sum() / len(net_attack.index)

In [ ]:
ml_net_attack = net_attack[NET_COLS]

In [ ]:
ml_net_attack

In [ ]:
lof_result = lof(ml_net_attack, contam)

In [ ]:
evaluate(net_attack["label_n"].iloc[:100000], lof_result)

In [ ]:
if_result = isolation_forest(ml_net_attack, contam)

In [ ]:
evaluate(net_attack["label_n"].iloc[:100000], if_result)

In [ ]:
svc(net_attack, NET_COLS)

In [ ]:
model_fit_network(DecisionTreeClassifier(), NET_COLS, net_attack)

In [ ]:
#model_fit_network(XGBClassifier(), NET_COLS, net_attack)
#XGBoost est pas content avec les donnees du network

## With Physical Data

In [ ]:
phy_attack

In [ ]:
contamination = (phy_attack["Label_n"] == 1).sum() / len(phy_attack.index) 

In [ ]:
lof_result = lof(phy_attack.loc[:, phy_attack.columns != "Label_n"], contamination=contamination, preprocess=False)

In [ ]:
evaluate(phy_attack["Label_n"], lof_result)

In [ ]:
if_result = isolation_forest(phy_attack.loc[:, phy_attack.columns != "Label_n"], contamination=contamination, preprocess=False)

In [ ]:
evaluate(phy_attack["Label_n"], if_result)

In [ ]:
svc_physical(phy_attack, phy_attack.loc[:, phy_attack.columns != "Label_n"], phy_attack["Label_n"])

In [ ]:
model_fit(DecisionTreeClassifier(), phy_attack, phy_attack.loc[:, phy_attack.columns != "Label_n"], phy_attack["Label_n"])

In [ ]:
model_fit(MLPClassifier(), phy_attack, phy_attack.loc[:, phy_attack.columns != "Label_n"], phy_attack["Label_n"])

In [ ]:
model_fit(XGBClassifier(), phy_attack, phy_attack.loc[:, phy_attack.columns != "Label_n"], phy_attack["Label_n"])

# TODO: 
  #### Better preprocessing of data in order to have a more accurate prediction (especially with the physical dataset).
  #### Trying the models on the other csv.
  #### Training the models with data from all csv files.
  #### Trying to aggregate both types of data.
  #### Classifiation of the type of attacks.
    